### **Storing on the NFS**

This notebook will show how to connect to an exisiting file system and write/read to that FS. To connect, a persistent volume claim has to be set up to you namespace. The files written to the NFS are permanent and passable between workflows.

**First some imports and global settings**

In [ ]:
from hera.workflows import models, script, DAG, Workflow
from hera.shared import global_config

global_config.host = "https://services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Setting up Volume**

As with the empty directories, the NFS has to be mounted as a volume. There has to be an existing pvc for our namespace to connect to the NFS. For our case we will connect to the EODC NFS. The security context has to be set according to the UserID and GroupID of the folder you want to write to.

In [ ]:
nfs_volume = models.Volume(
    name="eodc-mount",
    persistent_volume_claim={"claimName": "eodc-nfs-claim"},
    )

security_context = {"runAsUser": <YOUR UID>,
                    "runAsGroup": <YOUR GID>}

**Writing scripts**

Again, as with the empty directories, the volume has to be mounted in the script decorator. 

For our example we will write a simple csv to the eodc volume and read it in the next step.

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])
def write_to_volume():
    import pandas as pd

    data = {
    "A": [1, 2, 3, 4, 5],
    "B": [10, 20, 30, 40, 50],
    "C": [100, 200, 300, 400, 500]
    }

    df = pd.DataFrame(data)
    df.to_csv("/eodc/private/tempearth/data.csv")

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="eodc-mount", mount_path="/eodc")])
def read_from_volume():
    import pandas as pd

    df = pd.read_csv("/eodc/private/tempearth/data.csv")
    print(df)

**Creating the Workflow**

When creating the workflow, the volume and security context have to be passed in the respective parameters.

In [ ]:
with Workflow(
    generate_name="filesystem-access-",
    volumes = [nfs_volume],
    security_context=security_context,
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        create = write_to_volume()
        read = read_from_volume()

        create >> read

**Submitting the Workflow**

In [ ]:
w.create()